In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/options.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/financials.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/secondary_stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/trades.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jpx-tokyo-stock-exchange-prediction/jpx_tokyo_market_prediction/__init__.py
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_fin_spec.csv
/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/trades_spec.csv
/kaggle/input/jpx-tokyo-stock-

# Here starts my Baseline

Ideas:

1  - Feature engeneering- Basic and effective;

2 - Gradient Boosting Regressions;

3 - Neuronets Regressions; and

4 - Submission.



## 1 Feature Engeneering

In [2]:

def concat_df(df1, df2):
    df1 = pd.concat([df1, df2],
                    ignore_index=True, sort=False
                    ).drop_duplicates(["RowId"], keep="first")
    return df1

In [3]:
import gc

In [4]:
# For now... use pandas.... 
#If too heavy use Cudf

# For now... just fix the floats
dtypes = {
'SecuritiesCode' :     np.int16,
'Open'           :     np.float16,
'High'           :     np.float16,
'Low'            :     np.float16,
'Close'          :     np.float16,
'Volume'         :       np.int8,
'AdjustmentFactor':    np.float16,
'ExpectedDividend':    np.float16,
'Target'        :      np.float16
}
path = "../input/jpx-tokyo-stock-exchange-prediction/"
df_prices = pd.read_csv(f"{path}train_files/stock_prices.csv",dtype = dtypes)
#prices = pd.read_csv(f"{path}supplemental_files/stock_prices.csv",dtype = dtypes)
#df_prices = concat_df(df_prices, prices)
#prices = pd.read_csv(f"{path}train_files/secondary_stock_prices.csv",dtype = dtypes)
#df_prices = concat_df(df_prices, prices)
#prices = pd.read_csv(f"{path}supplemental_files/secondary_stock_prices.csv",dtype = dtypes)
#df_prices = concat_df(df_prices, prices)
df_prices = df_prices[df_prices.Date<"2021-06-01"]
df_prices.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2078531 entries, 0 to 2078530
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   RowId             2078531 non-null  object 
 1   Date              2078531 non-null  object 
 2   SecuritiesCode    2078531 non-null  int16  
 3   Open              2071589 non-null  float16
 4   High              2071589 non-null  float16
 5   Low               2071589 non-null  float16
 6   Close             2071589 non-null  float16
 7   Volume            2078531 non-null  int8   
 8   AdjustmentFactor  2078531 non-null  float16
 9   ExpectedDividend  16926 non-null    float16
 10  SupervisionFlag   2078531 non-null  bool   
 11  Target            2078293 non-null  float16
dtypes: bool(1), float16(7), int16(1), int8(1), object(2)
memory usage: 83.3+ MB


In [5]:
df_prices = df_prices[(df_prices['Date']<'2021-06-01' )& (df_prices['Date']>'2021-05-01')]

In [6]:
# def prep_prices(price):
#     price.fillna(0,inplace=True)
#     return price

In [7]:
# df_prices = prep_prices(df_prices)
# pd.options.display.float_format = '{:,.6g}'.format
# df_prices.describe()

In [8]:
# # Open stock_list
# stock_list = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")
# stock_list

In [9]:
# #Match Stocklist to securities code
# stock_list = stock_list[['SecuritiesCode','NewMarketSegment','33SectorCode','17SectorCode','Universe0','Section/Products','NewIndexSeriesSize']]
# stock_list = stock_list.replace(np.nan,'-')
# stock_list['Universe0'] = np.where(stock_list['Universe0'], 1, 0)
# stock_list = stock_list.drop_duplicates()
# stock_list


In [10]:
from datetime import datetime
import time

 # auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%Y-%m-%d").timetuple()))

# Data Features


In [11]:

# define function to compute log returns
def log_return_feature(series, periods=1):
    return np.log(series).diff(periods=periods)


# def fill_the_gaps(df):
#     new_df = pd.DataFrame(columns= df.columns)
#     for i in range(len(df['SecuritiesCode'].unique())):
#         new_df = new_df.append(df[df['SecuritiesCode'] == i].reindex(range(df[df['SecuritiesCode'] == i].index[0],df[df['SecuritiesCode'] == i].index[-1]+60,60),method='pad'))
#     new_df.fillna(method = 'pad', inplace = True) 
#     return new_df


def rsiFunc(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi



def get_features(df):

    df['log_return_5'] = log_return_feature(df['Close'],periods=5)
    df['log_return'] = log_return_feature(df['Close'],periods=1)

    upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
    lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

    df['upper_shadow'] = upper_shadow(df)
    df['lower_shadow'] = lower_shadow(df)
    
#     df['EMA_21'] = df['Close'].ewm(span=21).mean()
    
#     df['EMA_55'] = df['Close'].ewm(span=55).mean()
    
#     df['EMA_315'] = df['Close'].ewm(span=315).mean()
    
#     df['EMA_825'] = df['Close'].ewm(span=825).mean()
    
    window = 7
    
    no_of_std = 2
    
    df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
    df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
#     df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
#     df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
    window = 5
    
    df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
    df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
#     df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
#     df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
    df['MACD'] = df['EMA_7'] - df['EMA_5']
    
    
    df['rsi_5'] = rsiFunc(df['Close'].values, 5)
    
    df['rsi_7'] = rsiFunc(df['Close'].values, 7)
    
    df['rsi_21'] = rsiFunc(df['Close'].values, 21)
    
    
    
    df['VWAP'] = (df['Close'] * df['Volume'])/ df['Volume']




    
    
    #df = pd.concat([df, pd.get_dummies(df['Asset_ID'], prefix= 'Asset_')], axis=1)

    return df



In [12]:
df_prices = df_prices.sort_values('SecuritiesCode')

df_prices = get_features(df_prices)

df_prices = df_prices.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

df_prices = df_prices.fillna(0)

/opt/conda/lib/python3.7/site-packages/pandas/core/algorithms.py:1711: RuntimeWarning: invalid value encountered in subtract
  out_arr[res_indexer] = op(arr[res_indexer], arr[lag_indexer])
/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:1292: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


## Data Preparation



In [13]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [14]:
scaled_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'log_return_5', 'log_return', 'upper_shadow', 'lower_shadow',
       'VWAP', 'EMA_7', 'EMA_7_std','EMA_5', 'EMA_5_std', 'MACD', 'rsi_5', 'rsi_7', 'rsi_21']
features = ["SecuritiesCode",'Open', 'High', 'Low', 'Close', 'Volume', 'log_return_5', 'log_return', 'upper_shadow', 'lower_shadow',
       'VWAP', 'EMA_7', 'EMA_7_std','EMA_5', 'EMA_5_std', 'MACD', 'rsi_5', 'rsi_7', 'rsi_21']

In [15]:
df_prices[scaled_features] = scaler.fit_transform(df_prices[scaled_features])

## Data Prep 2

From Keras tutorial:
https://keras.io/examples/timeseries/timeseries_classification_transformer/

In [16]:
# # Creating endog and exog variables
y = df_prices[['Target']]
X = df_prices[features]

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [19]:
del df_prices
gc.collect()

21

## Build The Model

In [20]:
from tensorflow import keras
from tensorflow.keras import layers


In [21]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [22]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

## Train and evaluate

In [23]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)


2022-06-23 02:28:06.331517: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 19, 1)]      0                                            
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 19, 1)        7169        input_1[0][0]                    
                                                                 input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 19, 1)        0           multi_head_attention[0][0]       
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 19, 1)        2           dropout[0][0]                

2022-06-23 02:28:06.944673: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


302/302 [==============================] - 79s 243ms/step - loss: 56.0169 - sparse_categorical_accuracy: 0.0220 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0330
Epoch 2/200
302/302 [==============================] - 73s 242ms/step - loss: 0.2956 - sparse_categorical_accuracy: 0.0306 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0330
Epoch 3/200
302/302 [==============================] - 73s 242ms/step - loss: 0.0962 - sparse_categorical_accuracy: 0.0306 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0330
Epoch 4/200
302/302 [==============================] - 73s 243ms/step - loss: 0.0442 - sparse_categorical_accuracy: 0.0307 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0330
Epoch 5/200
302/302 [==============================] - 73s 243ms/step - loss: 0.0279 - sparse_categorical_accuracy: 0.0308 - val_loss: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0330
Epoch 6/200
302/302 [==============================] - 73s 243ms/step -

In [24]:
model.evaluate(x_test, y_test, verbose=1)

372/372 [==============================] - 14s 37ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 0.0327


[0.0, 0.03265993297100067]

In [25]:
model.save("tr_1.tf")

2022-06-23 02:42:57.636887: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


# Submission


In [26]:
# 